In [1]:
%load_ext tensorboard

In [2]:
import os

import tensorflow as tf
import datetime
import time 

from midi_utils import *
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# all_samples = []

# for dirpath, _, filenames in os.walk('OnlyPiano_TPB128'):
#     for File in filenames:
#         path = os.path.join(dirpath, File)
#         mid = mido.MidiFile(path)
#         try:
#             samples = midi_to_pianoroll(mid)
#             all_samples.append(samples)
#         except:
#             pass
# all_samples = np.array(all_samples, dtype=np.uint8)
# np.save('samples.npy', all_samples)

In [3]:
x = np.load('samples.npy')

In [4]:
X_train = x[:-18]
X_valid = x[-18:]

In [5]:
def rounded_accuracy(y_true, y_pred):
    return tf.keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [6]:
np.random.seed(42)

stacked_encoder = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[128, 512]),
    tf.keras.layers.Dense(1024, activation="selu"),
    tf.keras.layers.Dense(512, activation="selu"),
])
stacked_decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, activation="selu", input_shape=[512]),
    tf.keras.layers.Dense(128*512, activation="sigmoid"),
    tf.keras.layers.Reshape([128, 512])
])
stacked_ae = tf.keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=1.5), metrics=[rounded_accuracy])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

start_time = time.time()
history = stacked_ae.fit(X_train, X_train, epochs=100,
                         validation_data=(X_valid, X_valid)
                        )
print("--- %s seconds ---" % (time.time() - start_time))

Epoch 1/100
7/7 [==============================] - 3s 48ms/step - loss: -16.8648 - rounded_accuracy: 0.5119 - val_loss: -7.6843 - val_rounded_accuracy: 0.5106
Epoch 2/100
7/7 [==============================] - 0s 26ms/step - loss: -23.2491 - rounded_accuracy: 0.5445 - val_loss: -7.3230 - val_rounded_accuracy: 0.5449
Epoch 3/100
7/7 [==============================] - 0s 24ms/step - loss: -22.9297 - rounded_accuracy: 0.5523 - val_loss: -7.1733 - val_rounded_accuracy: 0.5422
Epoch 4/100
7/7 [==============================] - 0s 23ms/step - loss: -22.6230 - rounded_accuracy: 0.5557 - val_loss: -7.1689 - val_rounded_accuracy: 0.5435
Epoch 5/100
7/7 [==============================] - 0s 24ms/step - loss: -22.4634 - rounded_accuracy: 0.5611 - val_loss: -7.0136 - val_rounded_accuracy: 0.5450
Epoch 6/100
7/7 [==============================] - 0s 24ms/step - loss: -22.3782 - rounded_accuracy: 0.5647 - val_loss: -7.1700 - val_rounded_accuracy: 0.5610
Epoch 7/100
7/7 [=============================

7/7 [==============================] - 0s 22ms/step - loss: -17.7064 - rounded_accuracy: 0.7371 - val_loss: -4.4126 - val_rounded_accuracy: 0.7787
Epoch 53/100
7/7 [==============================] - 0s 22ms/step - loss: -17.6648 - rounded_accuracy: 0.7452 - val_loss: -4.2893 - val_rounded_accuracy: 0.7711
Epoch 54/100
7/7 [==============================] - 0s 22ms/step - loss: -17.5110 - rounded_accuracy: 0.7406 - val_loss: -4.0231 - val_rounded_accuracy: 0.7775
Epoch 55/100
7/7 [==============================] - 0s 22ms/step - loss: -17.4343 - rounded_accuracy: 0.7469 - val_loss: -3.5948 - val_rounded_accuracy: 0.7809
Epoch 56/100
7/7 [==============================] - 0s 22ms/step - loss: -15.2804 - rounded_accuracy: 0.7530 - val_loss: -3.4674 - val_rounded_accuracy: 0.7876
Epoch 57/100
7/7 [==============================] - 0s 23ms/step - loss: -15.2681 - rounded_accuracy: 0.7557 - val_loss: -3.5766 - val_rounded_accuracy: 0.7816
Epoch 58/100
7/7 [==============================] - 0

In [7]:
a = stacked_ae.predict(X_valid)

In [9]:
# unique, counts = np.unique(a, return_counts=True)
# dict(zip(unique, counts))

In [19]:
for i in range(10):
    mid = pianoroll_to_midi(a[i])
    mid.save(f'generatedSongs/music{i+1}.mid')

In [11]:
%tensorboard --logdir logs/fit3